# Pacotes

In [194]:
%pip install openpyxl

In [195]:
import pandas as pd
import glob
import os
 
import openpyxl

# 1) Adquirindo conjunto de dados

## 1.1) Leitura dos arquivos

### Escolhi utilizar os arquivos csv pela facilidade de leitura e poder utilizar chunks de dados

In [196]:
def get_csv_files(years: range = range(2000, 2001), directory: str = "../data", chunksize: int = 10000) -> list: 
    dataframes_list = []

    for year in years:
        file_name = f"GERACAO_USINA_{year}.csv"
        file_path = os.path.join(directory, file_name)

        print(f"\nLendo o arquivo {file_name}...")

        if glob.glob(file_path):
            # Leia o arquivo XLSX
            df = pd.read_csv(file_path, chunksize=chunksize, delimiter=";", encoding="utf-8", low_memory=False)

            # Adicione o DataFrame à lista
            dataframes_list.append(df)
        else:
            raise Exception(f"O arquivo {file_name} não foi encontrado.")
            

    return dataframes_list

In [197]:
try: 
    dataframes = get_csv_files(range(2000, 2021), "../data", 10000)
except Exception as e:
    print(f"Erro: {e}")


Lendo o arquivo GERACAO_USINA_2000.csv...

Lendo o arquivo GERACAO_USINA_2001.csv...

Lendo o arquivo GERACAO_USINA_2002.csv...

Lendo o arquivo GERACAO_USINA_2003.csv...

Lendo o arquivo GERACAO_USINA_2004.csv...

Lendo o arquivo GERACAO_USINA_2005.csv...

Lendo o arquivo GERACAO_USINA_2006.csv...

Lendo o arquivo GERACAO_USINA_2007.csv...

Lendo o arquivo GERACAO_USINA_2008.csv...

Lendo o arquivo GERACAO_USINA_2009.csv...

Lendo o arquivo GERACAO_USINA_2010.csv...

Lendo o arquivo GERACAO_USINA_2011.csv...

Lendo o arquivo GERACAO_USINA_2012.csv...

Lendo o arquivo GERACAO_USINA_2013.csv...

Lendo o arquivo GERACAO_USINA_2014.csv...

Lendo o arquivo GERACAO_USINA_2015.csv...

Lendo o arquivo GERACAO_USINA_2016.csv...

Lendo o arquivo GERACAO_USINA_2017.csv...

Lendo o arquivo GERACAO_USINA_2018.csv...

Lendo o arquivo GERACAO_USINA_2019.csv...

Lendo o arquivo GERACAO_USINA_2020.csv...


### 1.2) Lendo Dataframes por chunks

In [198]:
def reading_chunks(df, cols: list = ["din_instante", "id_subsistema", "val_geracao", "nom_tipousina"]):
    output = pd.DataFrame()

    for chunk in df:
        details = chunk[cols]
    
        display(details.head(10000))

        # output = output.append(details, igore_index=True)


In [199]:
cols = ["din_instante", "id_subsistema", "val_geracao", "nom_tipousina"]

reading_chunks(dataframes[0], cols)

,din_instante,id_subsistema,val_geracao,nom_tipousina
0,2000-01-01 00:00:00,N,2422.5,HIDROELÉTRICA
1,2000-01-01 00:00:00,NE,1995.6,HIDROELÉTRICA
2,2000-01-01 00:00:00,NE,13.0,HIDROELÉTRICA
3,2000-01-01 00:00:00,NE,202.5,HIDROELÉTRICA
4,2000-01-01 00:00:00,NE,269.8,HIDROELÉTRICA
...,...,...,...,...
9995,2000-01-04 18:00:00,NE,47.7,HIDROELÉTRICA
9996,2000-01-04 18:00:00,NE,221.1,HIDROELÉTRICA
9997,2000-01-04 18:00:00,NE,423.3,HIDROELÉTRICA
9998,2000-01-04 18:00:00,NE,305.2,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
10000,2000-01-04 18:00:00,NE,740.9,HIDROELÉTRICA
10001,2000-01-04 18:00:00,NE,210.0,HIDROELÉTRICA
10002,2000-01-04 18:00:00,S,707.8,HIDROELÉTRICA
10003,2000-01-04 18:00:00,S,755.4,HIDROELÉTRICA
10004,2000-01-04 18:00:00,S,892.4,HIDROELÉTRICA
...,...,...,...,...
19995,2000-01-08 12:00:00,S,652.8,HIDROELÉTRICA
19996,2000-01-08 12:00:00,S,92.2,HIDROELÉTRICA
19997,2000-01-08 12:00:00,S,77.9,HIDROELÉTRICA
19998,2000-01-08 12:00:00,S,248.2,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
20000,2000-01-08 12:00:00,S,227.5,HIDROELÉTRICA
20001,2000-01-08 12:00:00,S,219.5,HIDROELÉTRICA
20002,2000-01-08 12:00:00,S,179.3,HIDROELÉTRICA
20003,2000-01-08 12:00:00,S,96.4,HIDROELÉTRICA
20004,2000-01-08 12:00:00,S,34.0,HIDROELÉTRICA
...,...,...,...,...
29995,2000-01-12 06:00:00,S,213.9,HIDROELÉTRICA
29996,2000-01-12 06:00:00,S,292.4,TÉRMICA
29997,2000-01-12 06:00:00,S,17.0,TÉRMICA
29998,2000-01-12 06:00:00,S,22.0,TÉRMICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
30000,2000-01-12 06:00:00,S,20.8,TÉRMICA
30001,2000-01-12 06:00:00,S,50.0,HIDROELÉTRICA
30002,2000-01-12 06:00:00,S,353.4,TÉRMICA
30003,2000-01-12 06:00:00,S,109.2,TÉRMICA
30004,2000-01-12 06:00:00,S,252.0,TÉRMICA
...,...,...,...,...
39995,2000-01-16 00:00:00,SE,25.0,HIDROELÉTRICA
39996,2000-01-16 00:00:00,SE,45.0,HIDROELÉTRICA
39997,2000-01-16 00:00:00,SE,125.0,HIDROELÉTRICA
39998,2000-01-16 00:00:00,SE,6.0,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
40000,2000-01-16 00:00:00,SE,1081.3,HIDROELÉTRICA
40001,2000-01-16 00:00:00,SE,354.2,HIDROELÉTRICA
40002,2000-01-16 00:00:00,SE,4789.0,HIDROELÉTRICA
40003,2000-01-16 00:00:00,SE,50.9,HIDROELÉTRICA
40004,2000-01-16 00:00:00,SE,48.6,HIDROELÉTRICA
...,...,...,...,...
49995,2000-01-19 18:00:00,SE,224.1,HIDROELÉTRICA
49996,2000-01-19 18:00:00,SE,105.0,HIDROELÉTRICA
49997,2000-01-19 18:00:00,SE,39.9,HIDROELÉTRICA
49998,2000-01-19 18:00:00,SE,260.5,HIDROELÉTRICA


...

,din_instante,id_subsistema,val_geracao,nom_tipousina
950000,2000-12-16 11:00:00,SE,86.2,HIDROELÉTRICA
950001,2000-12-16 11:00:00,SE,53.5,HIDROELÉTRICA
950002,2000-12-16 11:00:00,SE,0.0,HIDROELÉTRICA
950003,2000-12-16 11:00:00,SE,38.3,HIDROELÉTRICA
950004,2000-12-16 11:00:00,SE,34.0,HIDROELÉTRICA
...,...,...,...,...
959995,2000-12-20 01:00:00,S,127.2,HIDROELÉTRICA
959996,2000-12-20 01:00:00,S,821.1,HIDROELÉTRICA
959997,2000-12-20 01:00:00,S,9.7,TÉRMICA
959998,2000-12-20 01:00:00,S,26.0,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
960000,2000-12-20 01:00:00,S,41.5,HIDROELÉTRICA
960001,2000-12-20 01:00:00,S,66.3,HIDROELÉTRICA
960002,2000-12-20 01:00:00,S,67.5,HIDROELÉTRICA
960003,2000-12-20 01:00:00,S,40.0,HIDROELÉTRICA
960004,2000-12-20 01:00:00,S,0.0,TÉRMICA
...,...,...,...,...
969995,2000-12-23 14:00:00,SE,123.0,HIDROELÉTRICA
969996,2000-12-23 14:00:00,SE,52.7,HIDROELÉTRICA
969997,2000-12-23 14:00:00,SE,38.0,HIDROELÉTRICA
969998,2000-12-23 14:00:00,SE,320.4,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
970000,2000-12-23 14:00:00,SE,84.0000,HIDROELÉTRICA
970001,2000-12-23 14:00:00,SE,413.4700,NUCLEAR
970002,2000-12-23 14:00:00,SE,1350.3236,NUCLEAR
970003,2000-12-23 14:00:00,SE,0.0000,TÉRMICA
970004,2000-12-23 14:00:00,SE,0.0000,TÉRMICA
...,...,...,...,...
979995,2000-12-27 04:00:00,NE,818.4000,HIDROELÉTRICA
979996,2000-12-27 04:00:00,NE,212.0000,HIDROELÉTRICA
979997,2000-12-27 04:00:00,S,245.0000,HIDROELÉTRICA
979998,2000-12-27 04:00:00,S,0.0000,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
980000,2000-12-27 04:00:00,S,482.9,HIDROELÉTRICA
980001,2000-12-27 04:00:00,S,73.0,HIDROELÉTRICA
980002,2000-12-27 04:00:00,S,63.6,HIDROELÉTRICA
980003,2000-12-27 04:00:00,S,271.8,HIDROELÉTRICA
980004,2000-12-27 04:00:00,S,10.3,TÉRMICA
...,...,...,...,...
989995,2000-12-30 17:00:00,SE,130.6,TÉRMICA
989996,2000-12-30 17:00:00,SE,31.8,HIDROELÉTRICA
989997,2000-12-30 17:00:00,SE,0.0,TÉRMICA
989998,2000-12-30 17:00:00,SE,24.0,HIDROELÉTRICA


,din_instante,id_subsistema,val_geracao,nom_tipousina
990000,2000-12-30 17:00:00,SE,135.0,TÉRMICA
990001,2000-12-30 17:00:00,SE,5406.9,HIDROELÉTRICA
990002,2000-12-30 17:00:00,SE,47.7,HIDROELÉTRICA
990003,2000-12-30 17:00:00,SE,39.0,HIDROELÉTRICA
990004,2000-12-30 17:00:00,SE,150.0,HIDROELÉTRICA
...,...,...,...,...
993551,2000-12-31 23:00:00,SE,600.0,HIDROELÉTRICA
993552,2000-12-31 23:00:00,SE,211.2,HIDROELÉTRICA
993553,2000-12-31 23:00:00,SE,0.0,HIDROELÉTRICA
993554,2000-12-31 23:00:00,SE,0.0,TÉRMICA
